# Combined Parameters

In [1]:
import qcodes as qc
import numpy as np 

from qcodes.instrument.parameter import ManualParameter
from qcodes.utils.validators import Numbers
from qcodes.loops import Loop

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : /home/vsts/.qcodes/logs/command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


Qcodes Logfile : /home/vsts/.qcodes/logs/201202-25810-qcodes.log


If you want to sweep multiple parameters at once qcodes offers the combine function.
You can combine any number of any kind paramter. 
We'll use a ManualParameter for this example.

In [2]:
p1 = ManualParameter('p1', vals=Numbers(-10, 10))
p2 = ManualParameter('p2', vals=Numbers(-10, 10))
p3 = ManualParameter('p3', vals=Numbers(-10, 10))
p4 = ManualParameter('p4', vals=Numbers(-10, 10))
# set to -1 so we get some data out
p4.set(-1)

##  Simple combined  parameters 

In [3]:
combined = qc.combine(p1, p2, p3, name='combined')

sweep_vals = np.array([[1, 1,1], [1, 1,1]])


In [4]:
# 2d loop with a inner loop over a combined parameter
loop = Loop(p1.sweep(0,10,1)).loop(combined.sweep(sweep_vals), delay=0.001).each(p4)

In [5]:
data = loop.get_data_set(name='testsweep')

In [6]:
data = loop.run()

Started at 2020-12-02 08:13:57


DataSet:
   location = 'data/2020-12-02/#001_testsweep_08-13-57'
   <Type>   | <array_id>   | <array.name> | <array.shape>
   Setpoint | p1_set       | p1           | (11,)
   Setpoint | combined_set | combined     | (11, 2)
   Measured | p4           | p4           | (11, 2)
   Measured | p1           | p1           | (11, 2)
   Measured | p2           | p2           | (11, 2)
   Measured | p3           | p3           | (11, 2)
Finished at 2020-12-02 08:13:57


The combined_set just stores the indices 

In [7]:
print(data.combined_set)

DataArray[11,2]: combined_set
array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]])


But the acutal set values are saved, but labeled as "measured"

In [8]:
data.p3

DataArray[11,2]: p3
array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]])

## Combine and aggregate parameters

If an aggregator function is given, the aggregated values are saved instead of the indices.

In [9]:
# define an aggregator function that takes as arguments the parameters you whish to aggegate
def linear(x,y,z):
    return x+y+z

In [10]:
combined = qc.combine(p1, p2, p3, name='combined', label="Sum", unit="a.u", aggregator=linear)

x_vals = np.linspace(1, 2, 2)
y_vals = np.linspace(1, 2, 2)
z_vals = np.linspace(1, 2, 2)



In [11]:
# 2d loop with a inner loop over a combined parameter
loop = Loop(p1.sweep(0,10,1)).loop(combined.sweep(x_vals, y_vals, z_vals), delay=0.001).each(p4)

In [12]:
data = loop.get_data_set(name='testsweep')

In [13]:
data = loop.run()

Started at 2020-12-02 08:13:57
DataSet:
   location = 'data/2020-12-02/#002_testsweep_08-13-57'
   <Type>   | <array_id>   | <array.name> | <array.shape>
   Setpoint | p1_set       | p1           | (11,)
   Setpoint | combined_set | combined     | (11, 2)
   Measured | p4           | p4           | (11, 2)
   Measured | p1           | p1           | (11, 2)
   Measured | p2           | p2           | (11, 2)
   Measured | p3           | p3           | (11, 2)
Finished at 2020-12-02 08:13:57


the combined_set now stores the aggregated values

In [14]:
print(data.combined_set)

DataArray[11,2]: combined_set
array([[3., 6.],
       [3., 6.],
       [3., 6.],
       [3., 6.],
       [3., 6.],
       [3., 6.],
       [3., 6.],
       [3., 6.],
       [3., 6.],
       [3., 6.],
       [3., 6.]])


In [15]:
# snapshot of the combined parameter
combined.snapshot()

OrderedDict([('__class__', 'qcodes.instrument.parameter.CombinedParameter'),
             ('unit', 'a.u'),
             ('label', 'Sum'),
             ('full_name', 'combined'),
             ('aggregator', '<function linear at 0x7f65e3d34170>'),
             ('p1',
              {'__class__': 'qcodes.instrument.parameter.ManualParameter',
               'full_name': 'p1',
               'value': 2.0,
               'raw_value': 2.0,
               'ts': '2020-12-02 08:13:57',
               'name': 'p1',
               'inter_delay': 0,
               'post_delay': 0,
               'label': 'p1',
               'vals': '<Numbers -10<=v<=10>',
               'unit': ''}),
             ('p2',
              {'__class__': 'qcodes.instrument.parameter.ManualParameter',
               'full_name': 'p2',
               'value': 2.0,
               'raw_value': 2.0,
               'ts': '2020-12-02 08:13:57',
               'name': 'p2',
               'inter_delay': 0,
               'post_d